In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import os
from xgboost import XGBClassifier
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()

'/home/lukas/Projects/dmc2019'

In [2]:
%run notebooks/utils.ipynb

## Linear SVC

In [7]:
svc = SVC(C=56.98164719395536, cache_size=8000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.002848504943774676, verbose=0)

## XGBoost

In [8]:
xgboost = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eta=0.1769182150877735, eval_metric='aucpr',
       gamma=1.8285912697052542, reg_lambda=0.4149772770711012,
       learning_rate=0.1, max_bin=254, max_delta_step=7.2556696256684035,
       max_depth=3, min_child_weight=1.0317712458399741, missing=None,
       n_estimators=445, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       scale_pos_weight=1, seed=None, silent=True,
       subsample=1, tree_method='gpu_hist', verbosity=2)

In [23]:
from sklearn.preprocessing import MinMaxScaler
#For simpler scaling
df_train_complete =  pd.read_csv("data/train.csv", sep="|")
df_train_complete = add_new_features(df_train_complete)
df_train_complete_x = df_train_complete.drop(['fraud'], axis=1)

#Data For Training
df_train = pd.read_csv("data/train_new.csv", sep="|")
df_val = pd.read_csv("data/val_new.csv", sep="|")
df_test = pd.read_csv("data/test.csv", sep="|")

#Adding new features
df_train = add_new_features(df_train)
df_val = add_new_features(df_val)
df_test = add_new_features(df_test)

#Splitting X and Y
df_train_x = df_train.drop(['fraud'], axis=1)
df_train_y = df_train.fraud

df_val_x = df_val.drop(['fraud'], axis=1)
df_val_y = df_val.fraud

df_test_x = df_test


#Defining a Scaler on the whole dataset
my_scaler = MinMaxScaler()
my_scaler.fit(df_train_complete_x.append(df_test, sort=False))

df_train_x_scaled = scale_dataframe(df_train_x, my_scaler)
df_val_x_scaled = scale_dataframe(df_val_x, my_scaler)
df_test_x_scaled = scale_dataframe(df_test_x, my_scaler)

/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Creating a validation set which contains predictions of the classifiers

In [26]:
#svc.fit(df_train_x_scaled, df_train_y)
#xgboost.fit(df_train_x, df_train_y)
d1 = df_train_x_scaled.copy()
d1['fraud'] = df_train_y

d3 = df_train_x.copy()
d3['fraud'] = df_train_y

d2 = df_val_x_scaled.copy()
d2['fraud'] = df_val_y

d4 = df_val_x.copy()
d4['fraud'] = df_val_y

res = test_classification(svc,d1,d2)

print("\n\n XGBoost:")
res2 = test_classification(xgboost, d3, d4)

Results Fix Split: 
DMC Score: 70  ---  Normalized DMC Score: 0.18617021276595744, 

Results Cross Validation: 
DMC Score: 55.0  ---  Normalized DMC Score: 0.1462836879432624 


 XGBoost:
Results Fix Split: 
DMC Score: 15  ---  Normalized DMC Score: 0.0398936170212766, 

Results Cross Validation: 
DMC Score: 47.0  ---  Normalized DMC Score: 0.125 


In [30]:
df5 = res['dataframe']
df5[df5.prediction != df5.fraud]

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedItems,fraud,prediction,probablity
97,0.4,0.952459,0.784278,0.818182,0.8,0.8,0.000555,0.000451,0.027273,1.000000,0,1,0.569
312,0.2,0.797814,0.398240,0.545455,0.9,0.2,0.000552,0.000273,0.021818,0.827586,1,0,0.621
353,0.0,0.503825,0.726273,0.454545,0.9,0.6,0.000740,0.000789,0.021645,0.689655,1,0,0.579


In [31]:
df6 = res2['dataframe']
df6[df6.prediction != df6.fraud]

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedItems,fraud,prediction,probablity
87,1,1486,57.13,4,1,1,0.015478,0.038445,0.173913,23.0,0,1,0.556
97,3,1744,78.42,9,8,4,0.017202,0.044966,0.300000,30.0,0,1,0.522
133,2,1819,60.15,0,7,3,0.016493,0.033068,0.000000,30.0,1,0,0.614
176,1,1625,89.93,11,3,0,0.011692,0.055342,0.578947,19.0,1,0,0.612
200,1,1538,60.68,8,6,5,0.012354,0.039454,0.421053,19.0,1,0,0.731
208,2,1641,51.34,10,6,2,0.015235,0.031286,0.400000,25.0,1,0,0.934
353,1,923,72.62,5,9,3,0.022752,0.078678,0.238095,21.0,1,0,0.723


In [ ]:
distance = [find_nearest_neighbor_index(x,df_X) for x in df_test2[:10].values]

In [ ]:
def classifiy_sample(sample_id, xgboost_classifier, lsvc_classifier):
    if sample.trustLevel >= 3:
        return 0
    else: 
        idx, distance = find_nearest_neighbor(df_test_x_scaled.iloc[sample_id],df_val_x_scaled)
        # If distance to knn is to big, classify them directly
        if distance > 0.15:
            xgb_pred = xgboost_classifier.predict([df_test_x.iloc[sample_id]])[0]
            xgb_prob = max(np.ravel(xgboost_classifier.predict_proba([df_val_x.iloc[0]])))
            
            lsvc_pred = lsvc_classifier.predict([df_test_x_scaled.iloc[sample_id]])[0]
            lsvc_prob = max(np.ravel(lsvc_classifier.predict_proba([df_val_x.iloc[0]])))
            # If both classified them equal, take one of both
            if xgb_pred == lsvc_pred:
                return xgb_pred
            
            #if classification is not equal, take the one with higher probability
            elif xgb_prob > lsvc_prob:
                return xgb_pred
            else: 
                return lsvc_pred
        # If distance is smaller than 0.15, use knn    
        else:    
            best_classifier = best_classifier_for_sample(idx, df_val_x_scaled)
            if best_classifier == "xgboost":
                return xgboost_classifier.predict([df_test_x.iloc[sample_id]])[0]
            
            elif best_classifier == "lsvc":
                return lsvc_classifier.predict([df_test_x_scaled.iloc[sample_id]])[0]
            
            elif best_classifier is None:
                return None

In [ ]:
np.array(distance).shape

In [ ]:
for idx,elem in enumerate(distance):
    df_X["scores_{}".format(idx)] = elem

In [ ]:
df_X.describe()

In [ ]:
df_X[df_X.scores_0==df_X.scores_0.min()]

In [ ]:
type(df_test2.iloc[0])